In [1]:
# import dependancies 
import requests
import json
import pandas as pd
import pymongo
from config import gkey

In [2]:
# import disc golf courses, isolate addresses
nc_courses_df = pd.read_csv("../data/nc_courses_df.csv")

In [6]:
# make a connection to MongoDB
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.disc_golf_db

# drop collection to prevent duplicates
db.nc_courses.drop()

# create new collection
collection = db.nc_courses

In [7]:
# set up for loop for google api call
for index, row in nc_courses_df.iterrows():
    address = row[9]
    
    # set up params   
    params = {
        "address" : address,
        "key" : gkey
    }

    # make api call, isolate results
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    response = requests.get(base_url, params=params)
    disc_golf_data = response.json()
    results = disc_golf_data["results"]
    
    # gather lat and lng
    try: 
        lat = results[0]['geometry']['location']['lat']
        lng = results[0]['geometry']['location']['lng']
    except:
        pass
    
    # create post to MongoDB 
    post = {
        "Disc Golf Course" : row[5],
        "# Holes" : row[2],
        "City" : row[3],
        "State" : row[8],
        "Postal Code": row[6],
        "Rating" : row[7],
        "Course Address" : row[9],
        "Latitude" : lat,
        "Longitude" : lng
    }

    collection.insert_one(post)